# Introduction

Jupyter notebook for the simplest model scratch. By convention, all exogenous variables are presented with a overline line (*e.g.* $\overline a$) and the parameters are represented by greek letters (*e.g.* $\alpha$). The code in this document is executable and is strongly recommend to do the follow to ensure that the output is corrected and updated:

Run time > Restart and run all...

## Loading libraries

In [1]:
#!pip install pysolve3

%config InlineBackend.figure_format = 'retina'

from SFC_Setup import *

# Analytical solution

In [2]:
base = model()
df = SolveSFC(base, time=1000)

In [3]:
base_eq = model()
SolveSFC(base_eq, time=1, table = False)
t = sp.Symbol('t')
initials = {
    key: base_eq.evaluate(key) for key in base_eq.parameters
}
initials.update({key: base_eq.evaluate(key) for key in base_eq.variables})

for i in base_eq.variables:
  globals()["_" + i] = sp.Function(i)
  
for i in base_eq.parameters:
  globals()[i] = sp.symbols(i, positive=True)
  globals()['infla'] = sp.symbols('infla')

In [68]:
# Adjustment for r

rn = sp.Symbol('rn', positive=True) # Normal profit rate
_r = sp.Function('r') # Effective profit rate

## General equations

In [5]:
Y = _C(t) + _I_t(t)
pprint(sp.Eq(_Y(t), Y))
C = _Cw(t) + _Ck(t)
pprint(sp.Eq(_C(t), C))
I = _I_f(t) + _I_h(t)
pprint(sp.Eq(_I_t(t), I))
Yk = _K_f(t)/v
pprint(sp.Eq(_Yk(t), Yk))
u = _Y(t)/_Yk(t)
pprint(sp.Eq(_u(t), u))
Z = _I_h(t)
pprint(sp.Eq(_Z(t), Z))
W = omega*_Y(t)
pprint(sp.Eq(_W(t), W))
K = _K_HD(t) + _K_f(t)
pprint(sp.Eq(_K(t), K))
Z = _Ck(t) + _I_h(t)
pprint(sp.Eq(_Z(t), Z))

Y(t) = C(t) + Iₜ(t)
C(t) = Ck(t) + Cw(t)
Iₜ(t) = I_f(t) + Iₕ(t)
        K_f(t)
Yk(t) = ──────
          v   
        Y(t)
u(t) = ─────
       Yk(t)
Z(t) = Iₕ(t)
W(t) = ω⋅Y(t)
K(t) = K_HD(t) + K_f(t)
Z(t) = Ck(t) + Iₕ(t)


## Workers

In [6]:
Cw = alpha*_W(t)
pprint(sp.Eq(_Cw(t), Cw))
YDw = _W(t)
pprint(sp.Eq(_YDw(t), YDw))
S_hw = _YDk(t) - _Cw(t)
pprint(sp.Eq(_S_hw(t), S_hw))
NFW_hw = _S_hw(t)
pprint(sp.Eq(_NFW_hw(t), NFW_hw))

Cw(t) = α⋅W(t)
YDw(t) = W(t)
S_hw(t) = -Cw(t) + YDk(t)
NFW_hw(t) = S_hw(t)


## Capitalists

In [8]:
Ck = R*_Z(t)
pprint(sp.Eq(_Ck(t), Ck))
dLk = _Ck(t)
pprint(sp.Eq(_Lk(t) - _Lk(t-1), dLk))
YDk = _FD(t) + rm*_M_h(t-1) - _rmo(t)*_MO(t-1) - _rl(t)*_Lk(t-1)
pprint(sp.Eq(_YDk(t), YDk))
S_hk = _YDk(t) - _Ck(t)
pprint(sp.Eq(_S_hk(t), S_hk))
dMO = _I_h(t)
pprint(sp.Eq(_MO(t) - _MO(t-1), dMO))
dM_h = _S_hk(t) + (_Lk(t) - _Lk(t-1))
pprint(sp.Eq((_M_h(t) - _M_h(t-1)), dM_h))
V_h = _M_h(t) + _K_HD(t)*_ph(t) - _MO(t) - _Lk(t)
pprint(sp.Eq(_V_h(t), V_h))
V_hr = _M_h(t) + _K_HD(t) - _MO(t) - _Lk(t)
pprint(sp.Eq(_V_hr(t), V_hr))
NFW_h = _S_hk(t) - _I_h(t)
pprint(sp.Eq(_NFW_h(t), NFW_h))
M_h = _S_hk(t) + (_Lk(t) - _Lk(t-1))
pprint(sp.Eq(_M_h(t), M_h))

Ck(t) = R⋅Z(t)
Lk(t) - Lk(t - 1) = Ck(t)
YDk(t) = rm⋅Mₕ(t - 1) + FD(t) - Lk(t - 1)⋅rl(t) - MO(t - 1)⋅rmo(t)
Sₕₖ(t) = -Ck(t) + YDk(t)
MO(t) - MO(t - 1) = Iₕ(t)
Mₕ(t) - Mₕ(t - 1) = Lk(t) - Lk(t - 1) + Sₕₖ(t)
Vₕ(t) = K_HD(t)⋅ph(t) - Lk(t) - MO(t) + Mₕ(t)
Vₕᵣ(t) = K_HD(t) - Lk(t) - MO(t) + Mₕ(t)
NFWₕ(t) = -Iₕ(t) + Sₕₖ(t)
Mₕ(t) = Lk(t) - Lk(t - 1) + Sₕₖ(t)


## Firms

In [18]:
I_f = _h(t)*_Y(t)
pprint(sp.Eq(_I_f(t), I_f))
dK_f = _I_f(t)
pprint(sp.Eq(_K_f(t) - _K_f(t-1), dK_f))
Lf = _I_f(t) - _FU(t) + _Lf(t-1)
pprint(sp.Eq(_Lf(t), Lf))
FT = _FU(t) + _FD(t)
pprint(sp.Eq(_FT(t), FT))
FU = gamma_F*(_FT(t) - _rl(t)*_Lf(t-1))
pprint(sp.Eq(_FU(t), FU))
FD = (1 - gamma_F)*(_FT(t) - _rl(t)*_Lf(t-1))
pprint(sp.Eq(_FD(t), FD))
h = _h(t-1)*gamma_u*(_r(t)-rn) + _h(t-1)
pprint(sp.Eq(_h(t), h))
r = (1-omega)*_u(t)/v
pprint(sp.Eq(_r(t), r))
NFW_f = _FU(t) - _I_f(t)
pprint(sp.Eq(_NFW_f(t), NFW_f))
V_f = _K_f(t) - _Lf(t)
pprint(sp.Eq(_V_f(t), V_f))

I_f(t) = Y(t)⋅h(t)
K_f(t) - K_f(t - 1) = I_f(t)
Lf(t) = -FU(t) + I_f(t) + Lf(t - 1)
FT(t) = FD(t) + FU(t)
FU(t) = γ_F⋅(FT(t) - Lf(t - 1)⋅rl(t))
FD(t) = (1 - γ_F)⋅(FT(t) - Lf(t - 1)⋅rl(t))
h(t) = γᵤ⋅(-rn + r(t))⋅h(t - 1) + h(t - 1)
       (1 - ω)⋅u(t)
r(t) = ────────────
            v      
NFW_f(t) = FU(t) - I_f(t)
V_f(t) = K_f(t) - Lf(t)


## Banks

In [19]:
L = _Lf(t) + _Lk(t)
pprint(sp.Eq(_L(t), L))
M = (_L(t) - _L(t-1)) + (_MO(t) - _MO(t-1)) + _M(t-1)
pprint(sp.Eq(_M(t), M))
rmo = (1+ spread_mo)*rm
pprint(sp.Eq(_rmo(t), rmo))
rl = (1+ spread_l)*rm
pprint(sp.Eq(_rl(t), rl))
V_b = _L(t) + _MO(t) - _M(t)
pprint(sp.Eq(_V_b(t), V_b))
NFW_b = _rl(t)*_L(t-1) + _rmo(t)*_MO(t-1) - rm*_M(t-1)
pprint(sp.Eq(_NFW_b(t), NFW_b))

L(t) = Lf(t) + Lk(t)
M(t) = L(t) - L(t - 1) + M(t - 1) + MO(t) - MO(t - 1)
rmo(t) = rm⋅(spreadₘₒ + 1)
rl(t) = rm⋅(spreadₗ + 1)
V_b(t) = L(t) - M(t) + MO(t)
NFW_b(t) = -rm⋅M(t - 1) + L(t - 1)⋅rl(t) + MO(t - 1)⋅rmo(t)


## Residential Investment

In [12]:
_own = sp.Function('own')

K_HS = _K_HD(t)
pprint(sp.Eq(_K_HS(t), K_HS))
Is = _I_h(t)
pprint(sp.Eq(_Is(t), Is))
dK_HD = _I_h(t)
pprint(sp.Eq(_K_HD(t) - _K_HD(t-1), dK_HD))
I_h = (1+_g_Z(t))*_I_h(t-1)
pprint(sp.Eq(_I_h(t), I_h))
K_k = _K_HD(t)/(_K(t))
pprint(sp.Eq(_K_k(t), K_k))
ph = (1+infla)*_ph(t-1)
pprint(sp.Eq(_ph(t), ph))
own = ((1+_rmo(t))/(1+infla))-1
pprint(sp.Eq(_own(t), own))
g_Z = phi_0 - phi_1*_own(t)
pprint(sp.Eq(_g_Z(t), g_Z))

K_HS(t) = K_HD(t)
Is(t) = Iₕ(t)
K_HD(t) - K_HD(t - 1) = Iₕ(t)
Iₕ(t) = (g_Z(t) + 1)⋅Iₕ(t - 1)
        K_HD(t)
Kₖ(t) = ───────
          K(t) 
ph(t) = (infla + 1)⋅ph(t - 1)
              rmo(t) + 1
own(t) = -1 + ──────────
              infla + 1 
g_Z(t) = φ₀ - φ₁⋅own(t)


### Stability condition

In [14]:
g = sp.Function('g')
gK = sp.Function('g_K')


def replacer(express):
    #print("\nReplacing the initial values.....")
    df = SolveSFC(model(), time=1)
    df = df.iloc[1, :]

    express = express.subs(alpha, df['alpha']).subs(
        omega, df['omega'])
    express = express.subs(un, df['un']).subs(
        gamma_u, df['gamma_u']).subs(rn, df['rm'])
    express = express.subs(
        infla, df['infla'])
    express = express.subs(phi_0, df['phi_0']).subs(
        phi_1,
        df['phi_1']).subs(rm, df['rm']).subs(
            spread_mo, df['spread_mo'])
    express = express.subs(rm, df['rm']).subs(
            spread_mo, df['spread_mo']).subs(v, df['v']).subs(R, df['R'])
    return express

In [43]:
EqY = Y - _Y(t)
EqY = EqY.subs(_C(t), C).subs(_Ck(t), Ck).subs(_Cw(t), Cw)
EqY = EqY.subs(_I_t(t), I).subs(_I_f(t), I_f)
EqY = EqY.subs(_W(t), W)
EqY = EqY.subs(_I_h(t), (1-R)*_Z(t))

EqY = sp.solve(EqY, _Y(t))[0].collect(alpha).collect(omega)
solY = EqY.subs(alpha,1)
pprint(cse(solY, optimizations='basic')[1], use_unicode=True)
print('dY/d alpha = ', EqY.diff(alpha))
print('dY/d omega = ', EqY.diff(omega))

print("\nGowth rate.....")
gY, h_, gz_ = sp.symbols('gY h gZ')
gY_ = alpha*omega * gY + R*(_Z(t) / _Y(t)) * _g_Z(t) + _h(t) * gY + _h(t) - _h(
    t - 1) + (1-R)*(_Z(t) / _Y(t)) * _g_Z(t) - gY
gY_ = gY_.subs(_g_Z(t), gz_).subs(_Y(t), solY).subs(_g_Z(t), gz_).subs(alpha,1)
gY_ = gY_.subs(_h(t) - _h(t - 1), h - _h(t - 1))
gY_ = gY_.subs(_r(t), r)
gY_ = u_r(gY_)
gY_ = sp.solve(gY_, gY)[0].collect(gz_)
pprint(cse(gY_, optimizations='basic')[1], use_unicode=True)

print('\nd gY/ d omega\n')
pprint(gY_.diff(omega))

print("For u = un")
pprint(sp.Eq(gY, gY_.subs(_u(t), un)))

⎡   -Z(t)    ⎤
⎢────────────⎥
⎣ω + h(t) - 1⎦
dY/d alpha =  omega*Z(t)/(alpha*omega + h(t) - 1)**2
dY/d omega =  alpha*Z(t)/(alpha*omega + h(t) - 1)**2

Gowth rate.....
⎡gZ⋅x₀ + rn⋅x₁ - x₁⋅r(t)⎤
⎢───────────────────────⎥
⎣           x₀          ⎦

d gY/ d omega

     gZ        gZ⋅(ω + h(t) - 1) + γᵤ⋅rn⋅h(t - 1) - γᵤ⋅h(t - 1)⋅r(t)
──────────── - ─────────────────────────────────────────────────────
ω + h(t) - 1                                    2                   
                                  (ω + h(t) - 1)                    
For u = un
     gZ⋅(ω + h(t) - 1) + γᵤ⋅rn⋅h(t - 1) - γᵤ⋅h(t - 1)⋅r(t)
gY = ─────────────────────────────────────────────────────
                          ω + h(t) - 1                    


### Stability conditon (I)

In [79]:
def u_r(express):
    express = express.subs(_u(t), _r(t)*v/(1-omega)).subs(un, rn*v/(1-omega))
    return express

own_ = sp.Symbol('own')
g_LR = u_r(gY)
pprint(sp.Eq(g(t), g_LR))

Equ = _u(t)*(g(t) - gK(t)) + _u(t-1)
Equ = u_r(Equ)

pprint(sp.Eq(_u(t), Equ))
g_K = u_r((_h(t)*_u(t))/v)
pprint(sp.Eq(gK(t), g_K))
Equ = _u(t)*(g(t) - g_K)
Equ = Equ.subs(g(t), gz_)
Equ = u_r(Equ)
pprint(sp.Eq(_u(t), Equ))

Eqh = _h(t)*gamma_u*(_u(t) - un)
Eqh = u_r(Eqh)
pprint(sp.Eq(_h(t), Eqh))

print('\nBuilding Jacobian matrix and evaluating at u = un\n')
J = sp.Matrix([
        [
            Eqh.diff(_h(t)).subs(_u(t), un).subs(_h(t-1), _h(t)).subs(_h(t), g_LR*v/un).subs(_r(t), rn).subs(un, rn*v/(1-omega)), 
            Eqh.diff(_r(t)).subs(_u(t), un).subs(_h(t-1), _h(t)).subs(_h(t), g_LR*v/un).subs(_r(t), rn).subs(un, rn*v/(1-omega))
        ], 
        [
            Equ.diff(_h(t)).subs(_u(t), un).subs(_h(t-1), _h(t)).subs(_h(t), g_LR*v/un).subs(_r(t), rn).subs(un, rn*v/(1-omega)), 
            Equ.diff(_r(t)).subs(_u(t), un).subs(_h(t-1), _h(t)).subs(_h(t), g_LR*v/un).subs(_r(t), rn).subs(un, rn*v/(1-omega))
        ]])
pprint(J)
print('\nDeterminant:\n')
det = J.det().simplify().factor().subs(gz_, sp.Symbol('g^*', positive=True)).subs(g_LR, sp.Symbol('g^*', positive=True))
pprint(det>0)
estability = sp.solveset(det>0, gamma_u, domain=sp.S.Reals)
print("\nStability condition\n")
pprint(estability)

print('\nTrace:\n')
tr = J.trace().simplify().factor().subs(gz_, sp.Symbol('g^*', positive=True)).subs(g_LR, sp.Symbol('g^*', positive=True))
pprint(tr<0)
# estability = sp.solve(tr)
estability = sp.solveset(tr<0, omega, domain=sp.S.Reals)
print("\nStability condition\n")
pprint(estability)

g(t) = gY
       v⋅(g(t) - g_K(t))⋅r(t)           
u(t) = ────────────────────── + u(t - 1)
               1 - ω                    
         h(t)⋅r(t)
g_K(t) = ─────────
           1 - ω  
         ⎛     h(t)⋅r(t)⎞     
       v⋅⎜gZ - ─────────⎟⋅r(t)
         ⎝       1 - ω  ⎠     
u(t) = ───────────────────────
                1 - ω         
          ⎛   rn⋅v   v⋅r(t)⎞     
h(t) = γᵤ⋅⎜- ───── + ──────⎟⋅h(t)
          ⎝  1 - ω   1 - ω ⎠     

Building Jacobian matrix and evaluating at u = un

⎡                 gY⋅γᵤ⋅v        ⎤
⎢   0             ───────        ⎥
⎢                    rn          ⎥
⎢                                ⎥
⎢   2                            ⎥
⎢-rn ⋅v       gY⋅v   v⋅(-gY + gZ)⎥
⎢────────  - ───── + ────────────⎥
⎢       2    1 - ω      1 - ω    ⎥
⎣(1 - ω)                         ⎦

Determinant:

            2    
g__*⋅γᵤ⋅rn⋅v     
───────────── > 0
          2      
   (ω - 1)       

Stability condition

⎧              ⎛            2    ⎞⎫
⎪              ⎜g__*⋅γᵤ

In [95]:
Eqh.diff(omega).subs(_r(t), rn)

0

In [125]:
EqhLR = g_LR*v/un
EqhLR = u_r(EqhLR)
EqhLR = EqhLR.simplify()
f = EqhLR/(1-omega)
f.simplify()

gY/rn